# 1) Set up 

* import necessary modules 
* locate filepath of data 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 2) Sample Data
* use pandas to turn csv into dataframe
* view first 5 rows to get a feel for the data



In [ ]:
# read data in from csv
heart_data = pd.read_csv("/kaggle/input/parkinsons-disease-speech-signal-features/pd_speech_features.csv")
heart_data.head() # display first 5 rows

# 3) Data Processing
* training data is used to train the model
* cross validation is used to make sure we're not overfitting the data
* test set is to make predictions on

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

# form X matrix of data and y vector of outcomes
X = heart_data.copy()
y = X.pop('class') # pop last column off from X 

# normalise X so all features are within similar range
preprocessor = make_column_transformer((StandardScaler(),list(X.columns)))

# split into training, cross validation set
X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, stratify=y, train_size=0.6) #stratify makes sure classes are represented evenly across splits

# split cross validation set into cross val and test set
X_valid, X_test, y_valid, y_test = \
    train_test_split(X_valid, y_valid, stratify=y_valid, train_size=0.8)

X_train = preprocessor.fit_transform(X_train)
X_valid = preprocessor.transform(X_valid)
X_test = preprocessor.transform(X_test)
print(X_train.shape)
input_shape = [X_train.shape[1]] # gives number of features == number of input nodes

# 4) Define Network Architecture
* 4 Dense hidden layers
* each dense layer is normalized
* each layer has 10% of nodes dropped out
* final layer uses sigmoid activation function --> 0 or 1

In [ ]:
# use keras to define network architecture
model = keras.Sequential([
    layers.Dense(70,activation="relu",input_shape=input_shape), # hidden layer 1 with 70 nodes and relu activation fnc
    layers.BatchNormalization(), # normalize hidden layer --> more stability
    layers.Dropout(0.1), # randomly drop 10% of nodes to prevent overfitting
    layers.Dense(70,activation="relu"), # same as before
    layers.BatchNormalization(), #same as before
    layers.Dropout(0.1),
    layers.Dense(40,activation="relu"), # same as before
    layers.BatchNormalization(), #same as before
    layers.Dropout(0.1),
    layers.Dense(30,activation="relu"), # same as before
    layers.BatchNormalization(), #same as before
    layers.Dropout(0.1),
    layers.Dense(1,activation="sigmoid") # binary output so only 1 output node, activation function is sigmoid
])


# 5) compile the model 
* using "adam" optimizing algorithm which uses stochastic gradient descent
* using cross entropy loss function which defines probablity
* also define accuracy metrics

In [ ]:
# compile model with optimizing algorithm, loss function and accuracy function
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["binary_accuracy"])

# 6) Train the model
* use early stopping call back to prevent overfitting

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.001,
    restore_best_weights=True,
)
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=10,
    epochs=500,
    callbacks=[early_stopping],
    verbose=0
)

history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

# 7) Use parameters learnt by model to make predictions on test set

In [ ]:
predictions = [round(x[0]) for x in model.predict(X_test)]
y = [x for x in y_test]
num_true = 0
total = len(predictions)
for i in range(len(predictions)):
    if predictions[i] == y[i]:
        num_true += 1
accuracy = (num_true/total)*100

print("Percent of correct predictions: {}%".format(accuracy))